In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from collections import namedtuple
from typing import List, Dict, Union, NamedTuple

import prextract.title_filter as title_filter
import fitz

In [3]:
PATH='2.pdf'
doc = fitz.open(PATH)
pnum = 1

p = doc[pnum]
p_width = p.MediaBox[2]


class TextBlockTrans(NamedTuple):
    x0: float
    y0: float
    x1: float
    y1: float
    text: str
    block_no: int
    page: int
    pwidth: float = None
    def __repr__(self):
        ret = []
        ret.append('TextBlockTrans')
        ret.append('\tbbox: ({}, {}, {}, {})'.format(*self[:4]))
        ret.append('\ttext: {}'.format(self.text))
        ret.append('\tblock_no: {}'.format(self[5]))
        ret.append('\tpage, pwidth: {}'.format(self[-2:]))
        return '\n'.join(ret)
    

TextBlockPaged = namedtuple('TextBlockTrans',
    'x0 y0 x1 y1 text block_no page',
    defaults=7*(None,)
)

In [4]:
text_blocks = p.getTextBlocks()
extract = p.getTextPage().extractDICT()['blocks']
text_blocks[4:6]

[(56.69187545776367,
  441.5146789550781,
  404.0799865722656,
  582.56103515625,
  'ORDEM DE SERVIÇO Nº 354, DE 28 DE NOVEMBRO DE 2018\nO SUBSECRETÁRIO DE ADMINISTRAÇÃO GERAL, DA SECRETARIA DE ESTADO DE\nPLANEJAMENTO, ORÇAMENTO E GESTÃO DO DISTRITO FEDERAL, Substituto, no uso de\nsuas atribuições regimentais e com fundamento no art. 67 da Lei nº 8.666, de 21 de junho de 1993,\ne no art. 41 do Decreto nº 32.598, de 15 de dezembro de 2010, e ainda, acatando as solicitações da\nárea competente, resolve:\nArt. 1º Retificar o item 7 do art. 1º da ORDEM DE SERVIÇO Nº 238, DE 11 DE SETEMBRO DE\n2018, publicada no DODF nº 173, de 11/09/2018. ONDE SE LÊ: RA-XII/Anexo Sede (JM e\nConselho), LEIA-SE: RA-XII Samambaia - Anexo da Administração (Junta Militar, Conselhos, Núcleo\nde Feira, Transporte e Gerências).\nArt. 2º Esta Ordem de Serviço entra em vigor na data de sua publicação.\nArt. 3º Revogam-se as disposições em contrário.\nNAUM ROSIVALDO DOS SANTOS',
  4,
  0),
 (56.69187545776367,
  592

In [5]:
_TRASH_EXPRESSIONS = [
    "SUMÁRIO",
    "DIÁRIO OFICIAL",
    "SEÇÃO (I|II|III)",
    "SEÇÃO",
]

_TRASH_COMPILED = re.compile('|'.join(_TRASH_EXPRESSIONS))


def is_bold(flags):
    return flags & 2 ** 4


def textBlock_topage(lis, page_width, pnum):
    """Given a text_block list (of tuples), return one with 
    two more values at the end, indicating `page number` and
    `page width`.
    """
    return [TextBlockTrans(*i[:-1], pnum, page_width) for i in lis]

def text_blocks_transform(text_blocks: List,
                          keep_page_width=True):
    lis = []
    for idx, tb in enumerate(text_blocks):
        p_num = tb[-2]
        p_num = tb.page
        
        
        p_width = tb[-1]
        p_width = tb.pwidth
        
        
        p_num *= 2
        x0, y0, x1, y1 = tb[:4]
        x0 = tb.x0
        
        p_num += int(x0 >( p_width / 2))
        if keep_page_width:            
            lis.append( TextBlockTrans( *(tb[:-2]), p_num, p_width ) )
        else:
            lis.append( TextBlockPaged( *(tb[:-2]), p_num, ) )
    return lis


def is_title_subtitle(span):
    return ((title_filter.BoldUpperCase.dict_text(span))
            and is_bold(span['flags'])
            and not re.search(_TRASH_COMPILED, span['text'])
            and 'calibri' not in span['font'].lower()
        )

def are_title_subtitle(span_list):
    return [is_title_subtitle(span) for span in span_list]

In [6]:
tb_paged = textBlock_topage(text_blocks, p_width, pnum)
# tb_paged[4:6];

In [7]:
print(tb_paged[0])
tb_trans = text_blocks_transform(tb_paged, keep_page_width=False)
# tb_trans[5:7]
# [i.text for i in drop_header_footer(sort_transformed(
#     tb_trans
# ))]

TextBlockTrans
	bbox: (56.666988372802734, 55.11705017089844, 765.406005859375, 70.56302642822266)
	text: PÁGINA 2
Nº 228, segunda-feira, 3 de dezembro de 2018
D i ário Oficial do Distrito Federal
	block_no: 0
	page, pwidth: (1, 814.9600219726562)


## VERIFICAR SE UM BLOCO É CANDIDATO A TÍTULO

In [8]:
import re
UPPER_REG = re.compile(r'[A-Z]')
def is_upper(text: str):
    return text.upper() == text

In [9]:
def get_block_spans(block):
    span_lis = [] 
    for line in block['lines']: 
        for span in line['spans']: 
            span_lis.append(span) 
    return span_lis

titles_idx = [idx for (idx, bl) in enumerate(tb_trans)
          if is_upper(bl.text)]


In [10]:
block_spans = {}
for i in titles_idx:
    block_spans[i] = get_block_spans(extract[i])

In [11]:
block_spans.keys()

dict_keys([2, 3, 8, 9, 10, 11, 12, 15])

In [12]:
for k, span_list in block_spans.items():
    if all(are_title_subtitle(span_list)):
        print('\n'.join((sp['text'] for sp in span_list)))
        print('\t----- {} -----\n'.format(k))

SECRETARIA DE ESTADO DE PLANEJAMENTO,
ORÇAMENTO E GESTÃO
	----- 2 -----

SUBSECRETARIA DE ADMINISTRAÇÃO GERAL
	----- 3 -----

SECRETARIA DE ESTADO DA CASA CIVIL,
RELAÇÕES INSTITUCIONAIS E SOCIAIS
	----- 15 -----



In [13]:
def reading_sort_tuple(lis):
    return sorted(lis, key=lambda x: (x.page, int(x.y0), x.x0))


# def sort_transformed(text_blocks_transformed: 
#     List[Union[TextBlockTrans, TextBlockPaged]]):
#     return sorted(text_blocks_transformed, key=trans_key)


def drop_dup_tbt(lis: List[TextBlockTrans]):
    """. This fun
    
    Sometimes, a span text apears multiple times, as if there exists
    multiple spans starting at the same point. Tihs function drops
    duplicate which matches this case.
    """
    dic = {}
    for tup in lis:
        dic[tuple([int(i) for i in tup[:2]])] = tup
    return list(dic.values())
ret = reading_sort_tuple(drop_dup_tbt(tb_trans))
print(ret[0])


def drop_header_footer(lis: List[tuple]):
    y0l = [ x.y0 for x in lis ]
    mi, ma = min(y0l), max(y0l)
    idx_mi, idx_ma = y0l.index(mi), y0l.index(ma)
    left, right = min(idx_mi, idx_ma), max(idx_mi, idx_ma)
    del lis[left]
    del lis[right-1]
    return lis



def mount_hierarchy(page: fitz.Page, pnum):
#     TODO : fazer para o documento inteiro
    p_width = page.MediaBox[2]
    
    text_blocks = page.getTextBlocks()
    extract = page.getTextPage().extractDICT()['blocks']

    tb_paged = textBlock_topage(text_blocks, p_width, pnum)    
    tb_trans = text_blocks_transform(tb_paged, keep_page_width=False)
    
    reading_sorted = reading_sort_tuple(drop_dup_tbt(tb_trans))
    
    cleaned_and_sorted = drop_header_footer(
        reading_sort_tuple(
        drop_dup_tbt(
            tb_trans
        )))

    hier = [('preambulo', [])]
    
    last_title = 'preambulo'

    for text_block in cleaned_and_sorted:
        spans = get_block_spans(extract[text_block.block_no])
        if all(are_title_subtitle(spans)):
            cpy = [i.copy() for i in spans]
            cpy = [(sp['text'], sp['bbox']) for sp in cpy]
            last_title = sorted(set(cpy),
                       key= lambda x:(x[1][1], x[1][0]))

            last_title = '\n'.join([i[0] for i in last_title])
            hier.append((last_title, []))
        else:
            hier[-1][1].append(text_block.text)
    return hier
            
            

TextBlockTrans(x0=56.666988372802734, y0=55.11705017089844, x1=765.406005859375, y1=70.56302642822266, text='PÁGINA 2\nNº 228, segunda-feira, 3 de dezembro de 2018\nD i ário Oficial do Distrito Federal', block_no=0, page=2)


In [14]:
h=mount_hierarchy(p, pnum)

In [15]:
h

[('preambulo', []),
 ('SECRETARIA DE ESTADO DA CASA CIVIL,\nRELAÇÕES INSTITUCIONAIS E SOCIAIS',
  ['PORTARIA N° 84, DE 28 DE NOVEMBRO DE 2018\nO SECRETÁRIO DE ESTADO DA CASA CIVIL, RELAÇÕES INSTITUCIONAIS E SOCIAIS DO\nDISTRITO FEDERAL, com fulcro artigo 105, da Lei Orgânica do Distrito Federal, e no artigo 255 da Lei\nComplementar nº 840, de 23 de dezembro de 2011, resolve:\nArt. 1º. Determinar o arquivamento do Processo Administrativo Disciplinar, instaurada pela Portaria nº\n215, de 28 de julho de 2017, publicada no DODF nº 147, de 31 de julho de 2017, constante nos autos do\nProcesso Administrativo nº 390.000.509/2007, acolhendo os fundamentos da Nota Técnica n° 416/2018 -\nAJL.\nArt. 2º Esta Portaria entra em vigor na data de sua publicação.\nSÉRGIO SAMPAIO CONTREIRAS DE ALMEIDA',
   'PORTARIA N° 85, DE 28 DE NOVEMBRO DE 2018\nO SECRETÁRIO DE ESTADO DA CASA CIVIL, RELAÇÕES INSTITUCIONAIS E SOCIAIS DO\nDISTRITO FEDERAL, com fulcro artigo 105, da Lei Orgânica do Distrito Federal, e 